In [3]:
import os
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction import text
import joblib

# ============================
# 1. Config e caricamento dati
# ============================

CORPUS_JSON_PATH = "../data/processed/nba_players_corpus.json"
MODELS_DIR = "../data/models"

os.makedirs(MODELS_DIR, exist_ok=True)

# Carica corpus
corpus_df = pd.read_json(CORPUS_JSON_PATH)
corpus_df["text_profile"] = corpus_df["text_profile"].fillna("")

print("Corpus shape:", corpus_df.shape)

# Verifica che la colonna Status esista
status_available = "Status" in corpus_df.columns

# ============================
# 2. Aggiunta stato al testo
# ============================

if status_available:
    corpus_df["status_text"] = corpus_df["Status"].replace({
        "Active": "giocatore attivo",
        "Retired": "giocatore ritirato"
    })

    # arricchiamo il testo rendendo lo stato ricercabile
    corpus_df["full_text"] = corpus_df["text_profile"] + " " + corpus_df["status_text"]
else:
    corpus_df["full_text"] = corpus_df["text_profile"]

# ============================
# 3. Stopwords e TF-IDF
# ============================

# Stopwords personalizzate italiane
stopwords_it = [
    "giocatore", "giocatori", "ruolo", "provenienza",
    "nba", "draft", "stagioni", "partita", "partite",
]

# Trasformiamo tutto in LISTA (come richiesto da sklearn)
STOPWORDS = list(text.ENGLISH_STOP_WORDS) + stopwords_it

vectorizer = TfidfVectorizer(
    lowercase=True,
    ngram_range=(1, 2),
    max_df=0.90,        
    min_df=1,            
    stop_words=STOPWORDS
)


tfidf_matrix = vectorizer.fit_transform(corpus_df["full_text"])
print("TF-IDF matrix shape:", tfidf_matrix.shape)


# ============================
# 4. Salvataggio index IR
# ============================

VECTORIZER_PATH = os.path.join(MODELS_DIR, "tfidf_vectorizer.joblib")
MATRIX_PATH = os.path.join(MODELS_DIR, "tfidf_matrix.joblib")
META_PATH = os.path.join(MODELS_DIR, "index_metadata.csv")

joblib.dump(vectorizer, VECTORIZER_PATH)
joblib.dump(tfidf_matrix, MATRIX_PATH)

meta_cols = [c for c in [
    "player_id", "Player", "DraftYear", "Pick", 
    "PickBand", "Status", "text_profile"
] if c in corpus_df.columns]

metadata_df = corpus_df[meta_cols].reset_index(drop=True)
metadata_df.to_csv(META_PATH, index=False)

print("Indice salvato in:")
print(" -", VECTORIZER_PATH)
print(" -", MATRIX_PATH)
print(" -", META_PATH)

# ============================
# 5. Funzione di ricerca
# ============================

def normalize_query(query):
    """Normalizza la query utente: sinonimi, forme diverse, stato, ruoli."""
    q = query.lower().strip()

    # --- Stato ---
    q = q.replace("in attività", "giocatore attivo")
    q = q.replace("attivo", "giocatore attivo")

    # --- Ruoli ---
    q = q.replace("guardia tiratrice", "shooting guard")
    q = q.replace("guardia", "guard")
    q = q.replace("ala grande", "power forward")
    q = q.replace("ala piccola", "small forward")
    q = q.replace("centro", "center")

    # --- Sinonimi di tiro ---
    q = q.replace("tiratrice", "tiratore")
    q = q.replace("ottimo da tre", "tiratore da 3")
    q = q.replace("forte da tre", "tiratore da 3")

    # --- Sinonimi difesa ---
    q = q.replace("forte in difesa", "buon difensore")
    q = q.replace("difensore forte", "buon difensore")

    # --- Sinonimi rimbalzo ---
    q = q.replace("forte a rimbalzo", "ottimo rimbalzista")
    q = q.replace("rimbalzi", "rimbalzista")

    return q



def search_players(query, top_k=10, active_only=False, retired_only=False):
    """
    Esegue ricerca TF-IDF + filtri:
    - active_only = restituisce solo giocatori in attività
    - retired_only = restituisce solo giocatori ritirati
    """
    
    query_clean = normalize_query(query)

    # vettorizza query
    query_vec = vectorizer.transform([query_clean])

    # similarità coseno
    sim_scores = cosine_similarity(query_vec, tfidf_matrix).ravel()

    # ordina
    order = np.argsort(sim_scores)[::-1]
    results = metadata_df.iloc[order].copy()
    results["similarity"] = sim_scores[order]
    results["score"] = (results["similarity"] * 100).round(1)

    # === FILTRI PER STATO ===
    if active_only:
        results = results[results["Status"] == "Active"]
    if retired_only:
        results = results[results["Status"] == "Retired"]

    return results.head(top_k)

# ============================
# 6. Test rapido
# ============================

query = "cerco una guardia tiratrice in attività"
results = search_players(query, top_k=5, active_only=True)

pd.set_option("display.max_colwidth", 200)

cols = [c for c in ["Player", "Status", "DraftYear", "Pick", "score", "text_profile"] if c in results.columns]

print("Query:", query)
display(results[cols])


Corpus shape: (8323, 7)
TF-IDF matrix shape: (8323, 22023)
Indice salvato in:
 - ../data/models\tfidf_vectorizer.joblib
 - ../data/models\tfidf_matrix.joblib
 - ../data/models\index_metadata.csv
Query: cerco una guardia tiratrice in attività


,Player,Status,DraftYear,Pick,score,text_profile
8189,E.J. Liddell,Active,2022,41,14.5,"Giocatore: E.J. Liddell. Proveniente da Ohio State. Selezionato al Draft NBA 2022 con la pick 41 (SecondRound). Tiro da 3: 23.1%, tiratore da 3 punti poco affidabile. Tiri liberi: 1.0%, tiratore d..."
8163,Mark Williams,Active,2022,15,14.4,"Giocatore: Mark Williams. Proveniente da Duke. Selezionato al Draft NBA 2022 con la pick 15 (FirstRound). Tiro da 3: 0.0%, tiratore da 3 punti poco affidabile. Tiri liberi: 74.9%, tiratore di libe..."
8264,Chris Livingston,Active,2023,58,14.4,"Giocatore: Chris Livingston. Proveniente da Kentucky. Selezionato al Draft NBA 2023 con la pick 58 (SecondRound). Tiro da 3: 9.1%, tiratore da 3 punti poco affidabile. Tiri liberi: 71.4%, tiratore..."
8312,Harrison Ingram,Active,2024,48,14.2,"Giocatore: Harrison Ingram. Proveniente da UNC. Selezionato al Draft NBA 2024 con la pick 48 (SecondRound). Tiro da 3: 0.0%, tiratore da 3 punti poco affidabile. Segna 0.7 punti a partita. Cattura..."
8196,Kendall Brown,Active,2022,48,14.2,"Giocatore: Kendall Brown. Proveniente da Baylor. Selezionato al Draft NBA 2022 con la pick 48 (SecondRound). Tiro da 3: 0.0%, tiratore da 3 punti poco affidabile. Tiri liberi: 60.0%, tiratore di l..."


Indexing & IR:
 viene costruito l’intero sistema di ricerca del talent scout. Il notebook genera un indice TF-IDF persistente (tfidf_vectorizer.joblib e tfidf_matrix.joblib) che rappresenta ogni giocatore tramite la sua descrizione testuale completa. In parallelo viene creato il file index_metadata.csv, che contiene per ogni atleta tutte le informazioni utili (Player, Pick, DraftYear, Status, full_text) perfettamente allineate all’indice. È inoltre definita la funzione search_players(), che trasforma una query in linguaggio naturale in un vettore TF-IDF, calcola la similarità coseno con tutti i profili e restituisce i giocatori più coerenti con la richiesta. Questo step costituisce la base del sistema di talent scouting, permettendo già ricerche come: “cerco una guardia tiratrice con ottime percentuali da 3 e buon rimbalzista” e ottenendo i profili più simili.